# SofaScore Euros Initial Data.

In [1]:
import requests
import json
import random
import typing

from datetime import datetime

In [2]:
def make_request(url: str):
    user_agent_list = [ 
	'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36', 
	'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36', 
	'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15', 
    ]
    user_agent = None
    
    for i in range(1, 4):
        user_agent = random.choice(user_agent_list)
    
    headers = {
        'User-Agent': user_agent
    }

    return requests.get(url, headers=headers)

## Links and Its Descriptions

In [3]:
links = {
    'tournament': 'https://www.sofascore.com/api/v1/unique-tournament/1',
    'tournament_seasons': 'https://www.sofascore.com/api/v1/unique-tournament/1/seasons',
    'tournament_groups': 'https://www.sofascore.com/api/v1/unique-tournament/1/season/56953/groups',
    'tournament_events': 'https://www.sofascore.com/api/v1/tournament/1688/season/56953/events'
}

## Extracting Tournament Details.

In [4]:
req = make_request(links['tournament'])

In [5]:
data = req.json()['uniqueTournament']

In [6]:
tournament_data = {
    'name': data['name'],
    'slug': data['slug'],
    'category': data['category']['name'],
    'category_slug': data['category']['slug'],
    'category_sofascore_id': data['category']['id'],
    'tournament_start_timestamp': datetime.fromtimestamp(data['startDateTimestamp']),
    'tournament_end_timestamp': datetime.fromtimestamp(data['endDateTimestamp']),
}

In [7]:
tournament_data

{'name': 'EURO',
 'slug': 'european-championship',
 'category': 'Europe',
 'category_slug': 'europe',
 'category_sofascore_id': 1465,
 'tournament_start_timestamp': datetime.datetime(2024, 6, 15, 0, 30),
 'tournament_end_timestamp': datetime.datetime(2024, 7, 15, 2, 30)}

## Getting the Tournament Seasons.

In [8]:
req = make_request(links['tournament_seasons'])

In [9]:
seasons_data = req.json()['seasons']

In [10]:
tournament_seasons = list()
for season in seasons_data:
    tmp_data = dict()
    tmp_data['name'] = season['name']
    tmp_data['year'] = season['year']
    tmp_data['sofascore_id'] = season['id']
    tournament_seasons.append(tmp_data)

In [11]:
tournament_seasons

[{'name': 'EURO 2024', 'year': '2024', 'sofascore_id': 56953},
 {'name': 'Euro Cup 2020', 'year': '2021', 'sofascore_id': 26542},
 {'name': 'European Championship 2016', 'year': '2016', 'sofascore_id': 11098},
 {'name': 'European Championship 2012', 'year': '2012', 'sofascore_id': 4136},
 {'name': 'Euro Cup 2008', 'year': '2008', 'sofascore_id': 1162},
 {'name': 'Euro Cup 2004', 'year': '2004', 'sofascore_id': 356},
 {'name': 'Euro Cup 2000', 'year': '2000', 'sofascore_id': 358},
 {'name': 'Euro Cup 1996', 'year': '1996', 'sofascore_id': 27053},
 {'name': 'Euro Cup 1992', 'year': '1992', 'sofascore_id': 27052},
 {'name': 'Euro Cup 1988', 'year': '1988', 'sofascore_id': 27051},
 {'name': 'Euro Cup 1984', 'year': '1984', 'sofascore_id': 27048},
 {'name': 'Euro Cup 1980', 'year': '1980', 'sofascore_id': 27046},
 {'name': 'Euro Cup 1976', 'year': '1976', 'sofascore_id': 27049},
 {'name': 'Euro Cup 1972', 'year': '1972', 'sofascore_id': 27050},
 {'name': 'Euro Cup 1968', 'year': '1968', 'so

## Fetching Tournament Groups.

In [12]:
req = make_request(f'https://www.sofascore.com/api/v1/unique-tournament/1/season/{tournament_seasons[0]["sofascore_id"]}/groups')

In [13]:
groups = req.json()['groups']

### fetching group events.

In [14]:
groups

[{'tournamentId': 1688, 'groupName': 'Group A'},
 {'tournamentId': 1685, 'groupName': 'Group B'},
 {'tournamentId': 1686, 'groupName': 'Group C'},
 {'tournamentId': 1687, 'groupName': 'Group D'},
 {'tournamentId': 50059, 'groupName': 'Group E'},
 {'tournamentId': 50060, 'groupName': 'Group F'}]

In [15]:
group_ids: typing.List[typing.Tuple[int, str]] = [tuple(x.values()) for x in groups]

In [16]:
req = make_request(f'https://www.sofascore.com/api/v1/tournament/{group_ids[0][0]}/season/56953/events')

In [17]:
tournament_events = req.json()['events']

In [18]:
tournament_matches = list()
for match in tournament_events:
    tmp_data = {}
    tmp_data['league_stage'] = { 
        'sofascore_id': match['tournament']['id'],
        'name': match['tournament']['name'],
        'slug': match['tournament']['slug']
    }
    tmp_data['tournament'] = {
        'name': match['tournament']['uniqueTournament']['name'],
        'slug': match['tournament']['uniqueTournament']['slug'],
        'id': match['tournament']['uniqueTournament']['id'],
    }
    tmp_data['league_season'] = { 
        'name': match['season']['name'],
        'year': match['season']['year'],
        'sofascore_id': match['season']['id']
    }
    tmp_data['match_sofascore_id'] = match['id']
    tmp_data['homeTeam'] = { 
        'name': match['homeTeam']['name'],
        'slug': match['homeTeam']['slug'],
        'nameCode': match['homeTeam']['nameCode'],
        'sofascore_id': match['homeTeam']['id'],
    }
    tmp_data['awayTeam'] = {
        'name': match['awayTeam']['name'],
        'slug': match['awayTeam']['slug'],
        'nameCode': match['awayTeam']['nameCode'],
        'sofascore_id': match['awayTeam']['id'],
    }
    tmp_data['startTimestamp'] = datetime.fromtimestamp(match['startTimestamp'])
    tournament_matches.append(tmp_data)

In [19]:
tournament_matches

[{'league_stage': {'sofascore_id': 1688,
   'name': 'Euro, Group A',
   'slug': 'uefa-euro-group-a'},
  'tournament': {'name': 'EURO', 'slug': 'european-championship', 'id': 1},
  'league_season': {'name': 'EURO 2024',
   'year': '2024',
   'sofascore_id': 56953},
  'match_sofascore_id': 11873905,
  'homeTeam': {'name': 'Germany',
   'slug': 'germany',
   'nameCode': 'GER',
   'sofascore_id': 4711},
  'awayTeam': {'name': 'Scotland',
   'slug': 'scotland',
   'nameCode': 'SCO',
   'sofascore_id': 4695},
  'startTimestamp': datetime.datetime(2024, 6, 15, 0, 30)},
 {'league_stage': {'sofascore_id': 1688,
   'name': 'Euro, Group A',
   'slug': 'uefa-euro-group-a'},
  'tournament': {'name': 'EURO', 'slug': 'european-championship', 'id': 1},
  'league_season': {'name': 'EURO 2024',
   'year': '2024',
   'sofascore_id': 56953},
  'match_sofascore_id': 11873897,
  'homeTeam': {'name': 'Hungary',
   'slug': 'hungary',
   'nameCode': 'HUN',
   'sofascore_id': 4709},
  'awayTeam': {'name': 'Swit